<a href="https://colab.research.google.com/github/enitin-agarwal/azureforaws/blob/master/MimiciiiSql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Link to Colab python code https://colab.research.google.com/drive/1Unlo5GKcmASL1MV_BC0YkOkwbRl-Y88C?usp=sharing
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()

In [ ]:
# create and test connection to the database MimicIII
client = bigquery.Client(project='physionet-data')

# load the dataset list
datasets = client.list_datasets()

# iterate the datasets list
for dataset in datasets:
    did = dataset.dataset_id
    # print the dataset name
    print('Dataset "{}" has the following tables: '.format(did))
    # iterate the tables on the dataset
    for table in client.list_tables(client.dataset(did)):
        # print the table name
        print('- {}'.format(table.table_id))

Dataset "eicu_crd_demo" has the following tables: 
- admissiondrug
- admissiondx
- allergy
- apacheapsvar
- apachepatientresult
- apachepredvar
- careplancareprovider
- careplaneol
- careplangeneral
- careplangoal
- careplaninfectiousdisease
- customlab
- diagnosis
- hospital
- infusiondrug
- intakeoutput
- lab
- medication
- microlab
- note
- nurseassessment
- nursecare
- nursecharting
- pasthistory
- patient
- physicalexam
- respiratorycare
- respiratorycharting
- treatment
- vitalaperiodic
- vitalperiodic
Dataset "mimiciii_clinical" has the following tables: 
- admissions
- callout
- caregivers
- chartevents
- cptevents
- d_cpt
- d_icd_diagnoses
- d_icd_procedures
- d_items
- d_labitems
- datetimeevents
- diagnoses_icd
- drgcodes
- icustays
- inputevents_cv
- inputevents_mv
- labevents
- microbiologyevents
- outputevents
- patients
- prescriptions
- procedureevents_mv
- procedures_icd
- services
- transfers
Dataset "mimiciii_demo" has the following tables: 
- admissions
- callout
- 

In [48]:
%%bigquery --project ultimate-balm-449016-q6
SELECT COUNT(DISTINCT SUBJECT_ID) AS total_cancer_patients
FROM `physionet-data.mimiciii_demo.diagnoses_icd`
WHERE ICD9_CODE BETWEEN '140' AND '209';



Query is running:   0%|          |

Downloading:   0%|          |

,total_cancer_patients
0,29


In [ ]:
%%bigquery --project ultimate-balm-449016-q6

SELECT DISTINCT ICD9_CODE
FROM `physionet-data.mimiciii_demo.d_icd_diagnoses`
ORDER BY ICD9_CODE;

SELECT ICD9_CODE, SHORT_TITLE, LONG_TITLE
FROM `physionet-data.mimiciii_demo.d_icd_diagnoses`
WHERE ICD9_CODE BETWEEN '140' AND '209'
ORDER BY ICD9_CODE;




Query is running:   0%|          |

Downloading:   0%|          |

,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,1400,Mal neo upper vermilion,"Malignant neoplasm of upper lip, vermilion border"
1,1401,Mal neo lower vermilion,"Malignant neoplasm of lower lip, vermilion border"
2,1403,"Mal neo upper lip, inner","Malignant neoplasm of upper lip, inner aspect"
3,1404,"Mal neo lower lip, inner","Malignant neoplasm of lower lip, inner aspect"
4,1405,"Mal neo lip, inner NOS","Malignant neoplasm of lip, unspecified, inner ..."
...,...,...,...
694,20881,Oth leuk uns cl w rmson,"Other leukemia of unspecified cell type, in re..."
695,20882,Oth leuk uns cl-relapse,"Other leukemia of unspecified cell type, in re..."
696,20890,Leuk NOS w/o achv rmsn,"Unspecified leukemia, without mention of havin..."
697,20891,Leukemia NOS w remission,"Unspecified leukemia, in remission"


In [45]:
%%bigquery --project ultimate-balm-449016-q6

WITH cancer_diagnoses AS (
    SELECT * FROM `physionet-data.mimiciii_demo.d_icd_diagnoses` AS d
)
SELECT d.LONG_TITLE AS cancer_type, COUNT(*) AS diagnosis_count
FROM `physionet-data.mimiciii_demo.diagnoses_icd` di
JOIN `physionet-data.mimiciii_demo.d_icd_diagnoses` d ON di.ICD9_CODE = d.ICD9_CODE
WHERE d.ICD9_CODE BETWEEN '140%' AND '209%'
GROUP BY d.LONG_TITLE
ORDER BY diagnosis_count DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,cancer_type,diagnosis_count
0,"Malignant neoplasm of liver, secondary",9
1,Secondary malignant neoplasm of lung,8
2,Secondary malignant neoplasm of bone and bone ...,6
3,Malignant neoplasm of other parts of bronchus ...,4
4,Secondary malignant neoplasm of retroperitoneu...,4
5,"Other malignant lymphomas, unspecified site, e...",4
6,Secondary malignant neoplasm of brain and spin...,4
7,Malignant neoplasm of cardia,3
8,Secondary and unspecified malignant neoplasm o...,3
9,Secondary and unspecified malignant neoplasm o...,2


In [30]:
%%bigquery --project ultimate-balm-449016-q6

WITH cancer_procedures AS (
    SELECT * FROM `physionet-data.mimiciii_demo.d_icd_procedures` AS p
),
cancer_icd_procedures AS(
  SELECT * FROM `physionet-data.mimiciii_demo.procedures_icd` AS pi
  )

SELECT p.LONG_TITLE AS procedure_type, COUNT(*) AS patient_count
FROM `physionet-data.mimiciii_demo.procedures_icd` pi
JOIN `physionet-data.mimiciii_demo.d_icd_procedures` p ON pi.ICD9_CODE = p.ICD9_CODE
WHERE CAST(p.ICD9_CODE as STRING) Between '140' and '209'  -- Assuming 'C%' covers cancer-related procedures
GROUP BY p.LONG_TITLE
ORDER BY patient_count DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,procedure_type,patient_count
0,Infusion of vasopressor agent,4
1,Other operations on extraocular muscles and te...,1
2,Other excision or destruction of lesion or tis...,1
3,Excision of lesion or tissue of cerebral meninges,1


In [32]:
%%bigquery --project ultimate-balm-449016-q6

SELECT pr.DRUG, COUNT(*) AS prescription_count
FROM `physionet-data.mimiciii_demo.prescriptions` pr
JOIN `physionet-data.mimiciii_demo.diagnoses_icd` di ON pr.HADM_ID = di.HADM_ID
WHERE di.ICD9_CODE Between '140%' and '209%'
GROUP BY pr.DRUG
ORDER BY prescription_count DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,DRUG,prescription_count
0,0.9% Sodium Chloride,277
1,D5W,260
2,Potassium Chloride,245
3,SW,205
4,NS,195
5,Magnesium Sulfate,180
6,Morphine Sulfate,161
7,Furosemide,143
8,HYDROmorphone (Dilaudid),136
9,Sodium Chloride 0.9% Flush,132


In [37]:
%%bigquery --project ultimate-balm-449016-q6

SELECT
    a.INSURANCE,
    AVG(COALESCE(proc.procedure_count, 0) * 500 + COALESCE(pres.prescription_count, 0) * 100) AS estimated_avg_expense
FROM `physionet-data.mimiciii_demo.admissions` a
LEFT JOIN (
    SELECT HADM_ID, COUNT(*) AS procedure_count
    FROM `physionet-data.mimiciii_demo.procedures_icd`
    GROUP BY HADM_ID
) proc ON a.HADM_ID = proc.HADM_ID
LEFT JOIN (
    SELECT HADM_ID, COUNT(*) AS prescription_count
    FROM `physionet-data.mimiciii_demo.prescriptions`
    GROUP BY HADM_ID
) pres ON a.HADM_ID = pres.HADM_ID
WHERE a.HADM_ID IN (
    SELECT DISTINCT HADM_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE ICD9_CODE BETWEEN '140' AND '209'
)
GROUP BY a.INSURANCE
ORDER BY estimated_avg_expense DESC;



Query is running:   0%|          |

Downloading:   0%|          |

,INSURANCE,estimated_avg_expense
0,Private,15640.0
1,Medicare,9385.0
2,Medicaid,6700.0
3,Government,3000.0


In [39]:
%%bigquery --project ultimate-balm-449016-q6

SELECT DISTINCT INSURANCE FROM `physionet-data.mimiciii_demo.admissions`;

Query is running:   0%|          |

Downloading:   0%|          |

,INSURANCE
0,Medicare
1,Private
2,Medicaid
3,Government


In [42]:
%%bigquery --project ultimate-balm-449016-q6

SELECT COUNT(*) AS readmitted_cancer_patients
FROM (
    SELECT SUBJECT_ID, COUNT(HADM_ID) AS admission_count
    FROM `physionet-data.mimiciii_demo.admissions`
    WHERE SUBJECT_ID IN (
        SELECT DISTINCT SUBJECT_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE ICD9_CODE between '140%' and '209%'
    )
    GROUP BY SUBJECT_ID
    HAVING COUNT(HADM_ID) > 1
) AS readmitted_patients;


Query is running:   0%|          |

Downloading:   0%|          |

,readmitted_cancer_patients
0,6


In [47]:
%%bigquery --project ultimate-balm-449016-q6

SELECT
    CASE
        WHEN a.HOSPITAL_EXPIRE_FLAG = 1 THEN 'Failed Treatment'
        ELSE 'Successful Treatment'
    END AS treatment_outcome,
    COUNT(DISTINCT a.SUBJECT_ID) AS total_patients
FROM `physionet-data.mimiciii_demo.admissions` a
WHERE a.HADM_ID IN (
    SELECT DISTINCT HADM_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE ICD9_CODE between '140%' and '209%'
)
GROUP BY treatment_outcome;


Query is running:   0%|          |

Downloading:   0%|          |

,treatment_outcome,total_patients
0,Successful Treatment,20
1,Failed Treatment,11


In [49]:
%%bigquery --project ultimate-balm-449016-q6
SELECT
    CASE
        WHEN a.HOSPITAL_EXPIRE_FLAG = 1 THEN 'Failed Treatment'
        ELSE 'Successful Treatment'
    END AS treatment_outcome,
    COUNT(DISTINCT a.SUBJECT_ID) AS total_patients  -- Count unique patients, not admissions
FROM `physionet-data.mimiciii_demo.admissions` a
JOIN `physionet-data.mimiciii_demo.diagnoses_icd` d ON a.HADM_ID = d.HADM_ID
WHERE d.ICD9_CODE BETWEEN '140' AND '209'
GROUP BY treatment_outcome;


Query is running:   0%|          |

Downloading:   0%|          |

,treatment_outcome,total_patients
0,Successful Treatment,20
1,Failed Treatment,11


In [50]:
%%bigquery --project ultimate-balm-449016-q6

SELECT COUNT(DISTINCT a.SUBJECT_ID) AS extra_patients_in_admissions
FROM `physionet-data.mimiciii_demo.admissions` a
WHERE a.SUBJECT_ID NOT IN (
    SELECT DISTINCT SUBJECT_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd`
    WHERE ICD9_CODE BETWEEN '140' AND '209'
);


Query is running:   0%|          |

Downloading:   0%|          |

,extra_patients_in_admissions
0,71


In [1]:
%%bigquery --project ultimate-balm-449016-q6

SELECT COUNT(DISTINCT d.SUBJECT_ID) AS extra_patients_in_diagnoses
FROM `physionet-data.mimiciii_demo.diagnoses_icd` d
WHERE d.SUBJECT_ID NOT IN (
    SELECT DISTINCT SUBJECT_ID FROM `physionet-data.mimiciii_demo.admissions`
);


Query is running:   0%|          |

Downloading:   0%|          |

,extra_patients_in_diagnoses
0,0


In [2]:
%%bigquery --project ultimate-balm-449016-q6

SELECT pr.DRUG, COUNT(*) AS successful_treatments
FROM `physionet-data.mimiciii_demo.prescriptions` pr
JOIN `physionet-data.mimiciii_demo.admissions` a ON pr.HADM_ID = a.HADM_ID
WHERE a.HOSPITAL_EXPIRE_FLAG = 0 AND pr.HADM_ID IN (
    SELECT DISTINCT HADM_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE ICD9_CODE between '140%' and '209%'
)
GROUP BY pr.DRUG
ORDER BY successful_treatments DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,DRUG,successful_treatments
0,Potassium Chloride,95
1,0.9% Sodium Chloride,94
2,5% Dextrose,80
3,Furosemide,63
4,Metoprolol Tartrate,56
5,SW,55
6,Magnesium Sulfate,50
7,D5W,47
8,HYDROmorphone (Dilaudid),46
9,Iso-Osmotic Dextrose,40


In [6]:
%%bigquery --project ultimate-balm-449016-q6

SELECT
    CASE
        WHEN EXTRACT(YEAR FROM a.ADMITTIME) - EXTRACT(YEAR FROM p.DOB) < 18 THEN '0-17'
        WHEN EXTRACT(YEAR FROM a.ADMITTIME) - EXTRACT(YEAR FROM p.DOB) BETWEEN 18 AND 40 THEN '18-40'
        WHEN EXTRACT(YEAR FROM a.ADMITTIME) - EXTRACT(YEAR FROM p.DOB) BETWEEN 41 AND 60 THEN '41-60'
        ELSE '60+'
    END AS age_group,
    COUNT(DISTINCT p.SUBJECT_ID) AS patient_count
FROM `physionet-data.mimiciii_demo.patients` p
JOIN `physionet-data.mimiciii_demo.admissions` a ON p.SUBJECT_ID = a.SUBJECT_ID
JOIN `physionet-data.mimiciii_demo.diagnoses_icd` d ON a.HADM_ID = d.HADM_ID
WHERE d.ICD9_CODE between '140%' and '209%'
GROUP BY age_group
ORDER BY patient_count DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,age_group,patient_count
0,60+,23
1,41-60,6


In [10]:
%%bigquery --project ultimate-balm-449016-q6

SELECT
    ETHNICITY, COUNT(*) AS total_patients
FROM `physionet-data.mimiciii_demo.admissions`
WHERE HADM_ID IN (
    SELECT DISTINCT HADM_ID FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE ICD9_CODE Between '140%' and '209%'
)
GROUP BY ETHNICITY
ORDER BY total_patients DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,ETHNICITY,total_patients
0,WHITE,26
1,UNKNOWN/NOT SPECIFIED,5
2,ASIAN,1
3,UNABLE TO OBTAIN,1
